### Set up and connect to database

In [1]:
# Import Matplot lib and pandas
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt

import pandas as pd

In [2]:
# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
engine = create_engine("sqlite:///../flaskapp/peep.sqlite", echo=False)

### Explore database

In [4]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['timeline']

In [5]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('timeline')
for c in columns:
    print(c['name'], c["type"])    

index BIGINT
tweet TEXT
date DATETIME


In [6]:
# Quickly view data in a list form
engine.execute('SELECT * FROM timeline LIMIT 10').fetchall()

[(0, 'OMGGGGGGGGGGGG huge dumbass moment lord help me lol;;;;;;;;;;;', '2019-09-01 07:10:42.000000'),
 (1, 'RT @BestVideosviral: Pup does a prank https://t.co/poFilVWDLS', '2019-07-22 07:36:27.000000'),
 (2, "RT @wolfhard: In honour of the AT finale getting nominated for an Emmy, here's some headcanon that I've been holding onto for a while.\n\nI h…", '2019-07-20 20:57:27.000000'),
 (3, 'RT @AntennaButt: My lotion is so strong that someone in the computer lab asked “is someone making s’mores” 💀💀', '2019-03-05 03:04:09.000000'),
 (4, '@Escamilla_Dani wow mood this was me two days ago', '2019-02-26 02:47:12.000000'),
 (5, 'RT @muqingmzhang: If you’re white and you made up a fake Chinese, Japanese, or Korean name for yourself that you put on all your social med…', '2019-02-17 10:08:29.000000'),
 (6, 'RT @milkygoddess: so apparently it b like that all the time', '2019-02-14 07:30:51.000000'),
 (7, 'Ok my scorpio moon is out of control', '2019-02-14 07:15:16.000000'),
 (8, '@leexjessica I b

### Reflect database and Query

In [7]:
# Reflect Database into ORM class

# Make sure there's a primary key in the DB
Base = automap_base()
Base.prepare(engine, reflect=True)
timeline = Base.classes.timeline

In [8]:
session = Session(engine)

In [9]:
stmt = session.query(timeline).\
    order_by(timeline.date.desc()).statement

In [10]:
query = "control"
tweets = session.query(timeline.tweet, timeline.date).filter(timeline.tweet.like(f'%{query}%')).all()
df = pd.read_sql_query(stmt, session.bind)

#### Refer to: https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

In [11]:
# Total number of RT
all_rt = df[df['tweet'].str.match('RT')]
total_rt = all_rt['tweet'].count()
total_rt

66

In [12]:
# Total number of RT
all_rt2 = df[df['tweet'].str.contains('RT')]
total_rt2 = all_rt2['tweet'].count()
total_rt2

66

In [13]:
# Total replies
all_replies = df[df['tweet'].str.contains('@')]
total_replies = all_replies['tweet'].count()
total_replies

191

In [14]:
from collections import Counter 

# Top 3 RT person
rt_dict = {}
for x in all_rt['tweet']:
    tt = x.split()
    if tt[1] not in rt_dict:
        rt_dict[tt[1]] = 1
    else:
        rt_dict[tt[1]] += 1

rt_df = pd.DataFrame(list(rt_dict.items()), columns=['username', 'count'])
rt_df.sort_values(by='count', ascending=False).head(3)

,username,count
9,@SonyAnimation:,2
57,@FirstWorldPains:,2
0,@BestVideosviral:,1


In [15]:
# split all the words up

# len(" ".join(df["tweet"]).split())

uncleanwords = " ".join(df["tweet"]).split()

words = []
for x in uncleanwords:
    if '@' not in x:
        words.append(x)


In [16]:
# Most common word to tweet

from collections import Counter 

# Counter(" ".join(df["tweet"]).split()).most_common(10)

# Need to exclude common words

import nltk
# nltk.download('stopwords')

stopwords1 = nltk.corpus.stopwords.words('english')

# add an exclude filter
stopwords2 = ['u', 'I', 'RT', 'ur', 'da', 'im', 'r', 'like', 'rly', 'bc', 'dis','ya','rn','got','n','the','The']

stopwords = stopwords1.extend(stopwords2)


words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords1) 

topwords = pd.DataFrame(words_except_stop_dist.most_common(10),
                    columns=['Word', 'Frequency']).set_index('Word')

topwords

,Frequency
Word,
2,16
LOL,10
one,8
omg,7
roommate,7
dog,6
kno,6
time,5
taught,5


In [17]:
# Most used hash tag
# uncleanwords = " ".join(df["tweet"]).split()

hashtags = []
for x in uncleanwords:
    if '#' in x:
        hashtags.append(x)
        
hashtagfreq = nltk.FreqDist(w for w in hashtags) 

tophashtags = pd.DataFrame(hashtagfreq.most_common(10),
                    columns=['Word', 'Frequency']).set_index('Word')
tophashtags

,Frequency
Word,
#skills,1
#SpiderVerse,1
#BluePlanet2,1
#NationalDogDay,1
#why,1
#SFThingToDo,1
#hotpeopleproblems,1
😎#slick,1


In [18]:
# Month with highest tweet count
from datetime import datetime

copydf = df
copydf.head()

,index,tweet,date
0,0,OMGGGGGGGGGGGG huge dumbass moment lord help m...,2019-09-01 07:10:42
1,1,RT @BestVideosviral: Pup does a prank https://...,2019-07-22 07:36:27
2,2,RT @wolfhard: In honour of the AT finale getti...,2019-07-20 20:57:27
3,3,RT @AntennaButt: My lotion is so strong that s...,2019-03-05 03:04:09
4,4,@Escamilla_Dani wow mood this was me two days ago,2019-02-26 02:47:12


In [19]:
# new_dates, new_times = zip(*[(d.date(), d.time()) for d in copydf['date']])
# copydf = copydf.assign(new_date=new_dates, new_time=new_times)

copydf['date']=pd.to_datetime(copydf['date'],format='%y-%m-%d %H:%M:%S')

# copydf.groupby(pd.Grouper(freq='m'))


# copydf.groupby(by=[copydf.date])

copydf['hour'] = copydf['date'].map(lambda x: x.strftime('%H'))
copydf['day'] = copydf['date'].map(lambda x: x.strftime('%d'))
copydf['month'] = copydf['date'].map(lambda x: x.strftime('%m'))
copydf['year'] = copydf['date'].map(lambda x: x.strftime('%Y'))
copydf.head()

,index,tweet,date,hour,day,month,year
0,0,OMGGGGGGGGGGGG huge dumbass moment lord help m...,2019-09-01 07:10:42,07,01,09,2019
1,1,RT @BestVideosviral: Pup does a prank https://...,2019-07-22 07:36:27,07,22,07,2019
2,2,RT @wolfhard: In honour of the AT finale getti...,2019-07-20 20:57:27,20,20,07,2019
3,3,RT @AntennaButt: My lotion is so strong that s...,2019-03-05 03:04:09,03,05,03,2019
4,4,@Escamilla_Dani wow mood this was me two days ago,2019-02-26 02:47:12,02,26,02,2019


In [60]:
cf = copydf.groupby(by=['year']).count()
byyear = cf
byyear

,index,tweet,date,hour,day,month
year,,,,,,
2014,1,1,1,1,1,1
2015,69,69,69,69,69,69
2016,55,55,55,55,55,55
2017,73,73,73,73,73,73
2018,70,70,70,70,70,70
2019,12,12,12,12,12,12


In [57]:
x = [{"year": int(ind),"count": int(byyear['tweet'][ind])} for ind in byyear.index]
x

[{'year': 2014, 'count': 1},
 {'year': 2015, 'count': 69},
 {'year': 2016, 'count': 55},
 {'year': 2017, 'count': 73},
 {'year': 2018, 'count': 70},
 {'year': 2019, 'count': 12}]

In [54]:
x = [(index, row['tweet']) for index, row in cf.iterrows()]
x

[('2014', 1),
 ('2015', 69),
 ('2016', 55),
 ('2017', 73),
 ('2018', 70),
 ('2019', 12)]

In [21]:
copydf.groupby(by=['month']).count()

,index,tweet,date,hour,day,year
month,,,,,,
01,24,24,24,24,24,24
02,25,25,25,25,25,25
03,11,11,11,11,11,11
04,23,23,23,23,23,23
05,22,22,22,22,22,22
06,16,16,16,16,16,16
07,18,18,18,18,18,18
08,18,18,18,18,18,18
09,35,35,35,35,35,35


In [22]:
# Day of the week with high tweet count
copydf.groupby(by=['day']).count()

,index,tweet,date,hour,month,year
day,,,,,,
01,18,18,18,18,18,18
02,7,7,7,7,7,7
03,8,8,8,8,8,8
04,7,7,7,7,7,7
05,9,9,9,9,9,9
06,5,5,5,5,5,5
07,8,8,8,8,8,8
08,8,8,8,8,8,8
09,9,9,9,9,9,9


In [23]:
# Hour with highest tweet count
copydf.groupby(by=['hour']).count()

,index,tweet,date,day,month,year
hour,,,,,,
00,19,19,19,19,19,19
01,14,14,14,14,14,14
02,18,18,18,18,18,18
03,29,29,29,29,29,29
04,27,27,27,27,27,27
05,10,10,10,10,10,10
06,24,24,24,24,24,24
07,16,16,16,16,16,16
08,12,12,12,12,12,12


In [29]:
# Average word length of tweets
allthetweets = list(df['tweet'])

# noRT= []
# for x in allthetweets:
#     if "RT" not in x:
#         noRT.append(x)

noRT= [x for x in allthetweets if "RT" not in x]

        
noRTlen = [len(x.split()) for x in noRT]
    

avgword = round((sum(noRTlen) / len(noRTlen)),2)
avgword

9.06

In [33]:
# Average character length of tweets
noRTlenbychar = [len(list(x)) for x in noRT]
avgchar = round((sum(noRTlenbychar) / len(noRTlenbychar)),2)
avgchar


50.61

In [ ]:
# Tweet with the most likes